In [1]:
import os
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.metric_spec import MetricSpec

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [3]:
path = "./data/"
data = pd.read_csv("./data/2018_4_30_08_33_test.csv",na_values=['NA','?'])

In [4]:
data.head()

,Time,Speed,Shift,Load,Acc _Total,RPM,Pitch,Acc _Lat,Passengers,Car_Load,AC_Con,Window_Open,Radio_volume,Rain_intensity,Visibility,welbeing,Rush,class
0,0.019,0.0,0,0.0,0.0,0.0,0.040442,0.0,0,3,0,5,4,0,10,6,4,1
1,0.026,0.0,0,0.0,0.0,0.0,0.041683,0.0,0,3,0,5,4,0,10,6,4,1
2,0.038,0.0,0,0.0,0.0,0.0,0.042932,0.0,0,3,0,5,4,0,10,6,4,1
3,0.047,0.0,0,0.0,0.0,0.0,0.044192,0.0,0,3,0,5,4,0,10,6,4,1
4,0.056,0.0,0,0.0,0.0,0.0,0.045459,0.0,0,3,0,5,4,0,10,6,4,1


In [5]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Time,100000.0,500.051426,288.702342,0.0190,2.500175e+02,500.055000,750.062500,1000.1000
Speed,100000.0,4.078027,5.585968,-0.2476,-2.072500e-15,0.201475,8.523500,20.6750
Shift,100000.0,0.788560,1.303445,0.0000,0.000000e+00,0.000000,2.000000,5.0000
Load,100000.0,5.862021,7.415070,-1.0000,2.400000e+00,2.700000,5.161500,56.8810
Acc _Total,100000.0,0.178825,0.723494,-3.4104,-2.107800e-02,0.138665,0.261250,4.3355
RPM,100000.0,1328.372887,783.903367,-1.0000,7.632600e+02,857.545000,1765.500000,4683.9000
Pitch,100000.0,0.968272,0.740926,-3.5726,6.568275e-01,0.962530,1.391000,4.5020
Acc _Lat,100000.0,0.025686,0.726152,-5.0457,-1.042500e-05,0.000000,0.002773,5.4014
Passengers,100000.0,0.000000,0.000000,0.0000,0.000000e+00,0.000000,0.000000,0.0000
Car_Load,100000.0,3.000000,0.000000,3.0000,3.000000e+00,3.000000,3.000000,3.0000


In [6]:
data.shape

(100000, 18)

In [7]:
X = data.drop('class',axis=1)
y = data['class']

In [8]:
ccf_train_data = "./data/2018_4_21_22_38_train.csv"
ccf_test_data = "./data/2018_4_30_08_33_test.csv"


def load_data(filepath):
    from numpy import genfromtxt

    csv_data = genfromtxt(filepath, delimiter=",", skip_header=1)
    data = []
    labels = []

    for d in csv_data:
        data.append(d[:-1])
        labels.append(d[-1])

    return np.array(data), np.array(labels)

train_dataset, train_labels = load_data(ccf_train_data)
test_dataset, test_labels = load_data(ccf_test_data)

predicted_lables = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300,).fit_predict(train_dataset)

In [9]:
train_pl = tf.placeholder("float", [None, 17])
test_pl = tf.placeholder("float", [17])

knn_prediction = tf.reduce_sum(tf.abs(tf.add(train_pl, tf.negative(test_pl))), axis=1)

pred = tf.argmin(knn_prediction, 0)

with tf.Session() as tf_session:
    missed = 0

    for i in range(len(test_dataset)):
        knn_index = tf_session.run(pred, feed_dict={train_pl: train_dataset, test_pl: test_dataset[i]})
        
        print("Predicted class {} -- True class {}".format(train_labels[knn_index], predicted_lables[i]))
        
        if train_labels[knn_index] != predicted_lables[i]:
            missed += 1

    tf.summary.FileWriter("./samples/article/logs", tf_session.graph)
    

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

...

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True class 1
Predicted class 1.0 -- True 

IndexError: index 24140 is out of bounds for axis 0 with size 24140

In [ ]:
print("Missed: {} -- Total: {}".format(missed, len(test_dataset)))

In [ ]:
accuracy = (1-(missed/len(test_dataset)))
print("Accuracy : " , accuracy*100)